<a href="https://colab.research.google.com/github/ParikshitGehlaut/GPT_using_Pytorch/blob/main/GPT_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
  text = f.read()
print(len(text))

227050


In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [7]:
chars = sorted(list(set(text)))
print(len(chars))
print(''.join(chars))
vocab_size = len(chars)

79

 !"#&'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZ[]abcdefghijklmnopqrstuvwxyz﻿


In [8]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[59, 60, 60, 1, 71, 59, 56, 69, 56]
hii there


In [16]:
torch.manual_seed(1337)

# Hyperparameter

block_size = 32 # maximum length of sequences that the model processes at one time.
batch_size = 16 # what is the maximum context length for predictions
n_embed = 512 #  as per research paper
learning_rate = 1e-3

In [20]:
data = torch.tensor(encode(text), dtype=torch.long)

# train test splitting
n = int(0.9 * len(text))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i + 1:i+block_size + 1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
# print(xb)
print('targets:')
print(yb.shape)
# print(yb)
print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        # print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([16, 32])
targets:
torch.Size([16, 32])
----


In [38]:
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size, n_embed):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) #(B, T, C)
    # print("targets shape: ", targets.shape)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self,idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # print("logits shape before ", logits.shape)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # print("logits shape", logits.shape)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size,n_embed).to(device)
# logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

torch.Size([512, 79])
tensor(4.7870, device='cuda:0', grad_fn=<NllLossBackward0>)


In [40]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for steps in range(5000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5269947052001953


In [41]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))


re ares;(B?"
uperspP#Win

e,"OPK7)yv5UmangoisWins s otle arllielouthe#﻿l an iByleras sanyZy owelseve
